# Inception : Optimization

In [1]:
import math
import numpy as np
import tensorflow as tf
import keras
from keras import applications, Model
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.metrics import make_scorer
from keras.utils.np_utils import to_categorical
import random
import sklearn
import pandas as pd
import warnings

warnings.simplefilter('ignore')

Using TensorFlow backend.


In [2]:
train_data_dir = 'Data/Training_Data'
validation_data_dir = 'Data/Validation_Data'
test_data_dir= 'Data/Test_Data'

training_features_file = 'Features/training_features_Inception.npy'
validation_features_file = 'Features/validation_features_Inception.npy'
top_weights_file = 'Weights/weights_Inception.h5'
model_file = 'Models/model_inception.h5'

train_labels_file = 'Labels/training_labels.npy'
validation_labels_file = 'Labels/validation_labels.npy'
test_labels_file = 'Labels/test_labels.npy'

img_width, img_height = 224, 224
NB_CLASSES = 11

In [3]:
train_labels = np.load(open(train_labels_file, 'rb'))
validation_labels = np.load(open(validation_labels_file, 'rb'))
test_labels = np.load(open(test_labels_file, 'rb'))

In [4]:
print('Training Data : ' + str(len(train_labels)) + ' Images')
print('Validation Data : ' + str(len(validation_labels)) + ' Images')
print('Test Data : ' + str(len(test_labels)) + ' Images')

Training Data : 307 Images
Validation Data : 74 Images
Test Data : 18 Images


## Converting images to feature vectors using weights from ImageNet

In [5]:
def images_to_feature_vectors(model, directory, batch_size, steps):
    
    datagen = ImageDataGenerator()
    
    generator = datagen.flow_from_directory(
        directory,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False) # Keep the data in the same order
    
    features = model.predict_generator(generator, steps, verbose=1) 
    
    return features

In [6]:
# Batch size has to be a multiple of the number of images  to keep our vectors consistents
training_batch_size = 1 # batch size for feature pre-training
validation_batch_size = 1 # batch size for feature pre-training

model = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
training_features = images_to_feature_vectors(model, train_data_dir, training_batch_size, len(train_labels) // training_batch_size)
validation_features = images_to_feature_vectors(model, validation_data_dir, validation_batch_size, len(validation_labels) // validation_batch_size)

Instructions for updating:
Colocations handled automatically by placer.
Found 307 images belonging to 11 classes.
307/307 [==============================] - 27s 88ms/step
Found 74 images belonging to 11 classes.
74/74 [==============================] - 6s 82ms/step


In [7]:
with open(training_features_file, 'wb') as file:
        np.save(file, training_features, allow_pickle = False)
with open(validation_features_file, 'wb') as file:
        np.save(file, validation_features, allow_pickle = False)

## Randomized Search CV training

In [8]:
def create_model(lr, decay, nn1, nn2, nn3, input_shape, output_shape):
    '''This is a model generating function so that we can search over neural net 
    parameters and architecture'''
    
    opt = keras.optimizers.Adam(lr=lr, decay=decay)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, input_dim = input_shape, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu'))
    model.add(BatchNormalization())
            
    model.add(Dense(output_shape, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [9]:
model = KerasClassifier(build_fn=create_model, epochs=16, verbose=1)

In [10]:
# Learning rate values
lr=[1e-2, 1e-3, 1e-4]
decay=[1e-6,1e-9,0]

# Number of neurons per layer
nn1=[4096,2048,1024]
nn2=[2048,1024,512]
nn3=[1000,500,200]

batch_size=[2048,1024,512]

In [11]:
train_data = np.load(open(training_features_file, 'rb'))
#train_data = train_data.reshape(train_data.shape[0],-1)
validation_data = np.load(open(validation_features_file, 'rb'))
#validation_data = validation_data.reshape(validation_data.shape[0],-1)
    
train_labels_onehot = to_categorical(train_labels,NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels,NB_CLASSES)  #One Hot Encoder

In [12]:
# dictionary summary
param_grid = dict(
                    lr=lr, decay=decay, nn1=nn1, nn2=nn2, nn3=nn3,
                    batch_size=batch_size,
                    input_shape=train_data.shape[1:], output_shape = (NB_CLASSES,)
                 )


grid = RandomizedSearchCV(estimator=model, cv=KFold(3), param_distributions=param_grid, 
                          verbose=20,  n_iter=10, n_jobs=1)

In [16]:
grid_result = grid.fit(train_data, train_labels_onehot)

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] output_shape=101, nn3=200, nn2=1024, nn1=4096, lr=0.001, input_shape=5, decay=1e-06, batch_size=2048 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Instructions for updating:
Use tf.cast instead.
Epoch 1/16
2114/2114 [==============================] - 22s 10ms/step - loss: 5.1505 - acc: 0.0237
Epoch 2/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.5874 - acc: 0.2167
Epoch 3/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.2660 - acc: 0.2696
Epoch 4/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.1233 - acc: 0.3023
Epoch 5/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.0248 - acc: 0.3259
Epoch 6/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.9086 - acc: 0.3567
Epoch 7/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.8035 - acc: 0.3751
Epoch 8/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.6899 - acc: 0.4016
Epoch 9/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.5944 - acc: 0.4182
Epoch 10/16
2114/2114 [==============================] - 9s 4ms/step - loss: 

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min remaining:    0.0s


Epoch 1/16
2114/2114 [==============================] - 20s 9ms/step - loss: 5.2049 - acc: 0.0147
Epoch 2/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.6633 - acc: 0.1887
Epoch 3/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.2684 - acc: 0.2668
Epoch 4/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.1194 - acc: 0.2881
Epoch 5/16
2114/2114 [==============================] - 9s 4ms/step - loss: 3.0068 - acc: 0.3155
Epoch 6/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.8588 - acc: 0.3562
Epoch 7/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.7515 - acc: 0.3713
Epoch 8/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.6535 - acc: 0.3955
Epoch 9/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.5664 - acc: 0.4092
Epoch 10/16
2114/2114 [==============================] - 9s 4ms/step - loss: 2.4865 - acc: 0.4484
Epoch 11/16
2114/2114 [=====

...

[Parallel(n_jobs=1)]: Done  19 out of  19 | elapsed: 61.3min remaining:    0.0s


Epoch 1/16
2114/2114 [==============================] - 18s 9ms/step - loss: 6.1858 - acc: 0.0412
Epoch 2/16
2114/2114 [==============================] - 7s 3ms/step - loss: 7.0385 - acc: 0.0596
Epoch 3/16
2114/2114 [==============================] - 7s 3ms/step - loss: 5.7707 - acc: 0.0747
Epoch 4/16
2114/2114 [==============================] - 7s 3ms/step - loss: 5.1177 - acc: 0.1036
Epoch 5/16
2114/2114 [==============================] - 7s 3ms/step - loss: 5.0689 - acc: 0.0922
Epoch 6/16
2114/2114 [==============================] - 7s 3ms/step - loss: 4.6284 - acc: 0.1102
Epoch 7/16
2114/2114 [==============================] - 7s 3ms/step - loss: 4.3481 - acc: 0.1168
Epoch 8/16
2114/2114 [==============================] - 7s 3ms/step - loss: 4.3381 - acc: 0.1126
Epoch 9/16
2114/2114 [==============================] - 7s 3ms/step - loss: 4.1661 - acc: 0.1400
Epoch 10/16
2114/2114 [==============================] - 7s 3ms/step - loss: 4.0987 - acc: 0.1599
Epoch 11/16
2114/2114 [=====

2114/2114 [==============================] - 3s 1ms/step - loss: 1.5560 - acc: 0.7162
Epoch 16/16
1057/1057 [==============================] - 4s 4ms/step
[CV]  output_shape=101, nn3=200, nn2=2048, nn1=1024, lr=0.0001, input_shape=5, decay=1e-06, batch_size=2048, score=0.004, total= 1.1min
[CV] output_shape=101, nn3=200, nn2=2048, nn1=1024, lr=0.0001, input_shape=5, decay=1e-06, batch_size=2048 
Epoch 1/16
2114/2114 [==============================] - 15s 7ms/step - loss: 5.2600 - acc: 0.0151
Epoch 2/16
2114/2114 [==============================] - 3s 2ms/step - loss: 3.5396 - acc: 0.2318
Epoch 3/16
2114/2114 [==============================] - 3s 1ms/step - loss: 3.0783 - acc: 0.3335
Epoch 4/16
2114/2114 [==============================] - 3s 1ms/step - loss: 2.8476 - acc: 0.3907
Epoch 5/16
2114/2114 [==============================] - 3s 1ms/step - loss: 2.6631 - acc: 0.4418
Epoch 6/16
2114/2114 [==============================] - 3s 1ms/step - loss: 2.4931 - acc: 0.4834
Epoch 7/16
2114/21

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 82.7min finished


Epoch 1/16
3171/3171 [==============================] - 16s 5ms/step - loss: 5.0564 - acc: 0.0224
Epoch 2/16
3171/3171 [==============================] - 3s 1ms/step - loss: 3.5319 - acc: 0.2157
Epoch 3/16
3171/3171 [==============================] - 3s 1ms/step - loss: 2.8032 - acc: 0.4056
Epoch 4/16
3171/3171 [==============================] - 3s 1ms/step - loss: 2.2595 - acc: 0.5746
Epoch 5/16
3171/3171 [==============================] - 3s 1ms/step - loss: 1.8148 - acc: 0.6982
Epoch 6/16
3171/3171 [==============================] - 3s 1ms/step - loss: 1.4618 - acc: 0.7997
Epoch 7/16
3171/3171 [==============================] - 3s 1ms/step - loss: 1.1583 - acc: 0.8764
Epoch 8/16
3171/3171 [==============================] - 3s 1ms/step - loss: 0.9117 - acc: 0.9262
Epoch 9/16
3171/3171 [==============================] - 3s 1ms/step - loss: 0.7109 - acc: 0.9644
Epoch 10/16
3171/3171 [==============================] - 3s 1ms/step - loss: 0.5514 - acc: 0.9842
Epoch 11/16
3171/3171 [=====

In [17]:
cv_results_df = pd.DataFrame(grid_result.cv_results_)
cv_results_df.to_csv('gridsearch_Inception.csv')

In [18]:
cv_results_df

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
0     160.906361      1.215342         2.035298        0.069349   
1     293.947517      0.827491         2.931484        0.117531   
2     310.711635      0.515874         3.228173        0.139815   
3     116.765649      0.845978         2.688265        0.348304   
4     166.962746      1.540708         3.492018        0.524037   
5     116.682970      0.543197         3.068732        0.081720   
6     123.698080      0.398950         3.486437        0.207097   
7      99.318020      0.970573         3.719592        0.198820   
8     168.245318      1.524959         4.409509        0.084696   
9      62.198568      0.730994         4.392888        0.214267   

  param_output_shape param_nn3 param_nn2 param_nn1 param_lr param_input_shape  \
0                101       200      1024      4096    0.001                 5   
1                101      1000       512      4096     0.01                 5   
2                101      1000      2048      4096   0.0001              2048   
3                101      1000       512      2048   0.0001              2048   
4                101      1000      1024      2048    0.001              2048   
5                101       200      1024      2048    0.001              2048   
6                101      1000      2048      2048     0.01              2048   
7                101      1000      1024      1024    0.001                 5   
8                101       500      1024      4096   0.0001                 5   
9                101       200      2048      1024   0.0001                 5   

  param_decay param_batch_size  \
0       1e-06             2048   
1       1e-06              512   
2       1e-09              512   
3       1e-06             1024   
4       1e-09              512   
5           0             1024   
6       1e-09             1024   
7           0              512   
8       1e-06             2048   
9       1e-06             2048   

                                              params  split0_test_score  \
0  {'output_shape': 101, 'nn3': 200, 'nn2': 1024,...           0.000000   
1  {'output_shape': 101, 'nn3': 1000, 'nn2': 512,...           0.000000   
2  {'output_shape': 101, 'nn3': 1000, 'nn2': 2048...           0.000946   
3  {'output_shape': 101, 'nn3': 1000, 'nn2': 512,...           0.002838   
4  {'output_shape': 101, 'nn3': 1000, 'nn2': 1024...           0.000946   
5  {'output_shape': 101, 'nn3': 200, 'nn2': 1024,...           0.000000   
6  {'output_shape': 101, 'nn3': 1000, 'nn2': 2048...           0.000000   
7  {'output_shape': 101, 'nn3': 1000, 'nn2': 1024...           0.000000   
8  {'output_shape': 101, 'nn3': 500, 'nn2': 1024,...           0.002838   
9  {'output_shape': 101, 'nn3': 200, 'nn2': 2048,...           0.003784   

   split1_test_score  split2_test_score  mean_test_score  std_test_score  \
0           0.000000           0.000000         0.000000        0.000000   
1           0.000946           0.000000         0.000315        0.000446   
2           0.000000           0.000000         0.000315        0.000446   
3           0.000000           0.000000         0.000946        0.001338   
4           0.000000           0.000946         0.000631        0.000446   
5           0.000000           0.000000         0.000000        0.000000   
6           0.000000           0.000000         0.000000        0.000000   
7           0.000000           0.000000         0.000000        0.000000   
8           0.000946           0.001892         0.001892        0.000772   
9           0.007569           0.002838         0.004730        0.002044   

   rank_test_score  
0                7  
1                6  
2                5  
3                3  
4                4  
5                7  
6                7  
7                7  
8                2  
9                1

In [19]:
grid_result.best_params_

{'output_shape': 101,
 'nn3': 200,
 'nn2': 2048,
 'nn1': 1024,
 'lr': 0.0001,
 'input_shape': 5,
 'decay': 1e-06,
 'batch_size': 2048}

In [13]:
# Parameters after RandomizedSearchCV

nn1 = 1024; nn2 = 1024; nn3 = 200
lr = 0.0001; decay=1e-06
batch_size = 2048

## Regularization

In [14]:
# Regularzation Parameters

dropout = 0.5
l1 = 0.0001
l2 = 0.0001

In [15]:
def model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                     
    model = Sequential()
    
    model.add(Flatten())
    model.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
            
    model.add(Dense(NB_CLASSES, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [16]:
train_labels_onehot = to_categorical(train_labels, NB_CLASSES)            #One Hot Encoder
validation_labels_onehot = to_categorical(validation_labels, NB_CLASSES)  #One Hot Encoder

In [17]:
top = model()

In [28]:
top.fit(train_data, train_labels_onehot,
              epochs=64,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels_onehot))

Train on 307 samples, validate on 74 samples
Epoch 1/64
307/307 [==============================] - 2s 6ms/step - loss: 27.8003 - acc: 0.6515 - val_loss: 27.8392 - val_acc: 0.5541
Epoch 2/64
307/307 [==============================] - 2s 6ms/step - loss: 27.6151 - acc: 0.6352 - val_loss: 27.6577 - val_acc: 0.5270
Epoch 3/64
307/307 [==============================] - 2s 6ms/step - loss: 27.4293 - acc: 0.6352 - val_loss: 27.4794 - val_acc: 0.5270
Epoch 4/64
307/307 [==============================] - 2s 5ms/step - loss: 27.2699 - acc: 0.6450 - val_loss: 27.2951 - val_acc: 0.5270
Epoch 5/64
307/307 [==============================] - 2s 5ms/step - loss: 27.0522 - acc: 0.6580 - val_loss: 27.1135 - val_acc: 0.5405
Epoch 6/64
307/307 [==============================] - 2s 5ms/step - loss: 26.8089 - acc: 0.6612 - val_loss: 26.9316 - val_acc: 0.5270
Epoch 7/64
307/307 [==============================] - 2s 6ms/step - loss: 26.6344 - acc: 0.6808 - val_loss: 26.7473 - val_acc: 0.5270
Epoch 8/64
307/30

Epoch 62/64
307/307 [==============================] - 2s 6ms/step - loss: 17.8838 - acc: 0.9674 - val_loss: 18.7671 - val_acc: 0.5811
Epoch 63/64
307/307 [==============================] - 2s 6ms/step - loss: 17.7907 - acc: 0.9414 - val_loss: 18.6497 - val_acc: 0.5676
Epoch 64/64
307/307 [==============================] - 2s 6ms/step - loss: 17.6851 - acc: 0.9577 - val_loss: 18.5336 - val_acc: 0.5676


In [29]:
top.save_weights(top_weights_file)

## Complete Model

In [30]:
def complete_model():
    opt = keras.optimizers.Adam(lr=lr)
    reg = keras.regularizers.l1_l2(l1=l1, l2=l2)
                                                 
    base = applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_width,img_height,3))
    
    top = Sequential()

    top.add(Flatten(input_shape=train_data.shape[1:]))
    top.add(Dense(nn1, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn2, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())
    top.add(Dense(nn3, activation='relu', kernel_regularizer=reg))
    top.add(BatchNormalization())            
    top.add(Dense(NB_CLASSES, activation='softmax'))
    top.load_weights(top_weights_file)
    top.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    
    
    model = Model(input= base.input, output= top(base.output))
    for layer in model.layers:
        layer.trainable = False
    model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'],)
    return model

In [31]:
model = complete_model()

datagen = ImageDataGenerator()

print("Model Summary")
print(model.summary())

Model Summary
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_189 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_195 (BatchN (None, 111, 111, 32) 96          conv2d_189[0][0]                 
__________________________________________________________________________________________________
activation_189 (Activation)     (None, 111, 111, 32) 0           batch_normalization_195[0][0]    
_______________________________________________________________________________________________

In [32]:
generator = datagen.flow_from_directory(
            test_data_dir,
            target_size=(img_width, img_height),
            batch_size=len(test_labels),
            class_mode=None,
            shuffle=False)
test_predictions = model.predict_generator(generator, 1, verbose=1) 

Found 18 images belonging to 11 classes.
1/1 [==============================] - 8s 8s/step


In [33]:
test_predictions = np.asarray(list(map(str,np.argmax(test_predictions,axis=1)))).reshape(-1,1)
test_labels = np.asarray(test_labels).reshape(-1,1)
test_acc, test_acc_op = tf.metrics.accuracy(test_labels, test_predictions)

In [34]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [35]:
test_accuracy = sess.run(test_acc_op)

In [36]:
print('\nTest accuracy : ' + str(test_accuracy))


Test accuracy : 0.5555556


In [37]:
model.save(model_file)